## 0. Intro

In [ ]:
###############################################################################
#This script uses a series of api requests to obtain the first and last versions 
#of each bill for a particular session of the CA legislature. The results are
#then decoded from their base64 output and exported to two CSVs. One for each
#of the two bill versions. These can then be compared for similarity
##############################################################################

## 1. Required libraries

In [1]:
import requests #for api
import urllib #^
import json #^
from __future__ import division
import math
import base64 #to decode output
import pandas as pd to export csv

## 2. Get the session list for the state you want to look at so you can choose which years to look at.

In [2]:
# set key
key="KEY"

# set base url
base_url="http://api.legiscan.com/?key=KEY&op=getSessionlist"

# set response format
response_format=".json"

In [3]:
#these search parameters are for california particularly
#any other state can be used by changing to its abbreviation
search_params = {"state":"ca"}
#making the request
sessions = requests.get(base_url, params=search_params)
#turn the request into text
sessiontxt= sessions.text
sessionlist=json.loads(sessiontxt)
print(sessionlist)
#from this choose which session you want for CA by session id. whichever year choose that session id

{'sessions': [{'special': 0, 'year_end': 2018, 'year_start': 2017, 'session_id': 1400, 'state_id': 5, 'name': '2017-2018 Regular Session', 'session_name': 'Regular Session'}, {'special': 0, 'year_end': 2016, 'year_start': 2015, 'session_id': 1120, 'state_id': 5, 'name': '2015-2016 Regular Session', 'session_name': 'Regular Session'}, {'special': 0, 'year_end': 2014, 'year_start': 2013, 'session_id': 993, 'state_id': 5, 'name': '2013-2014 Regular Session', 'session_name': 'Regular Session'}, {'special': 0, 'year_end': 2012, 'year_start': 2011, 'session_id': 82, 'state_id': 5, 'name': '2011-2012 Session', 'session_name': 'Regular Session'}, {'special': 0, 'year_end': 2010, 'year_start': 2009, 'session_id': 30, 'state_id': 5, 'name': '2009-2010 Session', 'session_name': 'Regular Session'}], 'status': 'OK'}


## 3. Get list of all bills for session chosen above

In [4]:
# set key
key="KEY"

# set base url
base_url="http://api.legiscan.com/?key=KEY&op=getMasterlist"

# set response format
response_format=".json"

In [7]:
#put chosen session number here
search_params = {"state":"ca",
                "session_id": "30"}
#make request
bill = requests.get(base_url, params=search_params)
#make bill request into text
bilist= bill.text
sbills['masterlist'].keys()
#turn keys into list
keymax= list(sbills['masterlist'].keys())
#rempve key session because it is not applicale here. Only need bill ids.
while 'session' in keymax: keymax.remove('session')
#turn ids into integers for indexing
keymax=map(int, keymax)
#print(keymax)
#find the max bill number for later indexing
sessionmax= max(keymax)
print(sessionmax)
#use session max further on to figure out what index to use

5096


## 4. Turn dictionary into list of just bill numbers for that session

In [6]:
#make text dictionaries
sbills=json.loads(bilist)
#moving down into this particular dictionary because the two keys before it are not useful
sbills2=sbills['masterlist']
#remove word session
sbills2.pop("session", None)
#makes dictionaries for each individual id number
sbills3 = {int(k):dict(v) for k,v in sbills2.items()}
#print(sbills3)

In [13]:
#make list of each of the bill ids which will have the ids for the texts of each bill draft 

list1=[]
#this loops through the dictionary made above to pull out just the text identifier
for i in range(5097):
    num = sbills3[i]['bill_id']
    list1.append(num)
#print(list1)

AttributeError: 'list' object has no attribute 'encode'

## 5. Turn list of bills into dictionary of search parameters for looping later

In [31]:
#this loop creates a list of individual dictionaries
#it uses "the api search param: bill number"
spdict=[]
#makes an empty lis to append dictionaries to below
for item in list1[0:5097]:
    dict1= {"id":item}
    spdict.append(dict1)   
#print(spdict)

[{'id': 660996}, {'id': 660960}, {'id': 661065}, {'id': 661052}, {'id': 661001}, {'id': 661026}, {'id': 661043}, {'id': 661003}, {'id': 661018}, {'id': 660974}, {'id': 661062}, {'id': 786330}, {'id': 786563}, {'id': 660975}, {'id': 787111}, {'id': 660979}, {'id': 660995}, {'id': 787110}, {'id': 787679}, {'id': 660984}, {'id': 661039}, {'id': 787711}, {'id': 660964}, {'id': 787690}, {'id': 660992}, {'id': 787683}, {'id': 661012}, {'id': 789461}, {'id': 661019}, {'id': 786964}, {'id': 661063}, {'id': 786963}, {'id': 661059}, {'id': 661064}, {'id': 787676}, {'id': 661025}, {'id': 787672}, {'id': 787692}, {'id': 660986}, {'id': 660998}, {'id': 787695}, {'id': 661068}, {'id': 787708}, {'id': 661041}, {'id': 787688}, {'id': 661005}, {'id': 787710}, {'id': 660989}, {'id': 661031}, {'id': 661029}, {'id': 661020}, {'id': 661008}, {'id': 660971}, {'id': 661010}, {'id': 661066}, {'id': 661067}, {'id': 660978}, {'id': 661034}, {'id': 661056}, {'id': 661028}, {'id': 660966}, {'id': 661024}, {'id': 

## 6. Now do another request in order to get text id number for each bill.

In [15]:
# set key
key="KEY"

# set base url
base_url="http://api.legiscan.com/?key=KEY&op=getBill"

# set response format
response_format=".json"

In [17]:
#this gets a list of all the text ids for each bill number ---this includes intro, amend, enrolled, etc
#can then use text ids to loop through and get the text for each text id that matters
#this takes a very long time
billsum=[]
for i in spdict:
    #sets the parameters for the api pull
    r = requests.get(base_url, params=i)
    #turn into text and load json
    response_text= r.text
    sums=json.loads(response_text)
    #index to the text id's for each bill pulled
    texts= sums['bill']['texts']
    #make a new dictionary for each text id
    texts_dict = {}
    for k,v in [(key,d[key]) for d in texts for key in d]:
        if k not in texts_dict: texts_dict[k]=[v]
        else: texts_dict[k].append(v)
#texts_dict now appends to bill sum as a lost of dictionaries
    billtxtnum= texts_dict['doc_id']
    billsum.append(billtxtnum)

#print(billsum)

## 7. Pull out the first and last drafts of each bill to compare later

In [40]:
#get the first draft of thh bill (introduced)
#make empty list
introtext=[]
#loop through list of bills and pull out the first bill (indexed at 0)
for i in billsum:
    introtext.append(i[0])

#print(introtext)

[1052482, 1052448, 1052550, 1052537, 1052487, 1052512, 1052527, 1052489, 1052504, 1052461, 1052547, 1251169, 1252041, 1052462, 1254747, 1052466, 1052481, 1254746, 1255922, 1052471, 1052523, 1255954, 1052452, 1255933, 1052479, 1255926, 1052498, 1258976, 1052505, 1253842, 1052548, 1253841, 1052544, 1052549, 1255919, 1052511, 1255914, 1255935, 1052473, 1052484, 1255938, 1052553, 1255951, 1052525, 1255931, 1052491, 1255953, 1052476, 1052516, 1052514, 1052506, 1052494, 1052458, 1052496, 1052551, 1052552, 1052465, 1052519, 1052541, 1052513, 1052454, 1052510, 1052517, 1052472, 1052543, 1052456, 1052475, 1052508, 1052509, 1052493, 1052469, 1052521, 1052453, 1052780, 1052779, 1052778, 1054397, 1054769, 1055426, 1055424, 1055993, 1055844, 1056026, 1056025, 1056024, 1056867, 1056861, 1056858, 1056852, 1056864, 1056869, 1059663, 1059664, 1059665, 1059656, 1059655, 1059658, 1059653, 1059661, 1060674, 1060680, 1060673, 1060677, 1060676, 1062191, 1061904, 1062067, 1061945, 1061863, 1062027, 1062108, 

In [27]:
#do the exact same to get the last draft indexed at -1
lasttext=[]
for i in billsum:
    lasttext.append(i[-1])
#check length to double check that it looped correctly
#len(lasttext)

5097

## 8. Turn the first and last bills into a list of dictionaries to loop through the API

In [62]:
#make empty list to append dictionaries to
textdictI=[]
#loop through the list of intro drafts and add id for search params requirement 
for item in introtext[0:5097]:
    dicti= {"id":item}
    textdictI.append(dicti)
#print(textdictI)

#do the same for the ;ast draft fo the bill
textdictL=[]

for item in lasttext[0:5097]:
    dictl= {"id":item}
    textdictL.append(dictl)
#print(textdictL)

[{'id': 1052482}, {'id': 1052448}, {'id': 1052550}, {'id': 1052537}, {'id': 1052487}, {'id': 1052512}, {'id': 1052527}, {'id': 1052489}, {'id': 1052504}, {'id': 1052461}, {'id': 1052547}, {'id': 1251169}, {'id': 1252041}, {'id': 1052462}, {'id': 1254747}, {'id': 1052466}, {'id': 1052481}, {'id': 1254746}, {'id': 1255922}, {'id': 1052471}, {'id': 1052523}, {'id': 1255954}, {'id': 1052452}, {'id': 1255933}, {'id': 1052479}, {'id': 1255926}, {'id': 1052498}, {'id': 1258976}, {'id': 1052505}, {'id': 1253842}, {'id': 1052548}, {'id': 1253841}, {'id': 1052544}, {'id': 1052549}, {'id': 1255919}, {'id': 1052511}, {'id': 1255914}, {'id': 1255935}, {'id': 1052473}, {'id': 1052484}, {'id': 1255938}, {'id': 1052553}, {'id': 1255951}, {'id': 1052525}, {'id': 1255931}, {'id': 1052491}, {'id': 1255953}, {'id': 1052476}, {'id': 1052516}, {'id': 1052514}, {'id': 1052506}, {'id': 1052494}, {'id': 1052458}, {'id': 1052496}, {'id': 1052551}, {'id': 1052552}, {'id': 1052465}, {'id': 1052519}, {'id': 105254

## 9. Do a new request with the two draft dictionary lists

In [63]:
# set key
key="KEY"

# set base url
base_url="http://api.legiscan.com/?key=KEY&op=getBillText"

# set response format
response_format=".json"

In [70]:
#this makes the list of base64 code for the bill text
introbilltext64=[]
#for loop loops through request with the list of dictionaries for the first draft
for i in textdictI:
    #set request parameters
    req = requests.get(base_url, params=i)
    intro_text= req.text
    intro=json.loads(intro_text)
    #index into the request texts to get the text for the first draft (base64 encoded)
    textsi= intro['text']['doc']
    introbilltext64.append(textsi)
#index into the second level of the dictionaries to doc

#introbilltext64

In [89]:
#this gets base64 encoded text for the last draft of each bill
#set empty list to append base64 encoded texs to 
lastbilltext64=[]
for i in textdictL:
    #make request for api 
    requ = requests.get(base_url, params=i)
    last_text= requ.text
    last=json.loads(last_text)
    #index in to get just the base64 encoded document text
    textsl= last['text']['doc']
    lastbilltext64.append(textsl)
#lastbilltext64

## 10. Decode the base64 output from the API

In [84]:
#turn the base64 code of the bill text into actual text for analysis
#introbilldecode is a list of the text as strings 
import base64
#package base 64 decodes the base 64 string given by the api
introbilltextdecode = []
#set empty list to append to 
#for loop decodes each of the first draft bills
for i in introbilltext64:
    textcode= str(base64.b64decode(i))
    introbilltextdecode.append(textcode)
#introbilltextdecode

In [91]:
import base64
#set empty list to append all the decoded bills' last texts to 
lastbilltextdecode = []
#decode the base64 versions of the last bill texts
for i in lastbilltext64:
    textcode2= str(base64.b64decode(i))
    lastbilltextdecode.append(textcode2)
    
#lastbilltextdecode

## 11. Export th decoded bill texts to a CSV (one bill per row)

In [108]:
#this exports the list of decoded texts to a csv where each bill is a row in a colum
import pandas as pd
df = pd.DataFrame(introbilltextdecode, columns=["intro text"])
#new csv named introtext
#column named intro text and each row is a bill
df.to_csv('introtext.csv', index=False)

In [105]:
#this exports the list of decoded texts to a csv where each bill is a row in a colum
import pandas as pd
df2 = pd.DataFrame(lastbilltextdecode, columns=["last text"])
#new csv named lasttext
#column named last text and each row is a bill
df2.to_csv('lasttext.csv', index=False)